# Skrypt do predykcji danych demograficznych oparty o dane historyczne GUS

## Zaimportowanie bibliotek

In [51]:
import requests as rq

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

## Zaimportowanie danych z API GUS

Zaimportowanie danych z API GUS Portal z API BDL.

In [52]:
data = {
    "Area": [
        "Polska", "Malopolskie", "Slaskie", "Lubuskie", "Wielkopolskie",
        "Zachodniopomorskie", "Dolnoslaskie", "Opolskie", "Kujawsko-Pomorskie",
        "Pomorskie", "Warminsko-Mazurskie", "Lodzkie", "Swietokrzyskie",
        "Lubelskie", "Podkarpackie", "Podlaskie", "Mazowieckie"
    ],
    "Code": [
        "000000000000", "011200000000", "012400000000", "020800000000",
        "023000000000", "023200000000", "030200000000", "031600000000",
        "040400000000", "042200000000", "042800000000", "051000000000",
        "052600000000", "060600000000", "061800000000", "062000000000",
        "071400000000"
    ]
}

data2 = {
    "category_code": [
        "ludnosc_na_1_km2",
        "zmiana_liczby_ludnosci_na_1000_mieszkancow",
        "ludnosc_w_tysiacach",
        "wskaznik_urbanizacji",
        "liczba_ludnosci_ogolem",
        "liczba_ludnosci_ogolem_mezczyzni",
        "liczba_ludnosci_ogolem_kobiety",
        "mediana_wieku_ludnosci_ogolem",
        "mediana_wieku_ludnosci_mezczyzni",
        "mediana_wieku_ludnosci_kobiety",
        "ludnosc_w_wieku_nieprodukcyjnym_na_100_osob_w_wieku_produkcyjnym",
        "ludnosc_w_wieku_poprodukcyjnym_na_100_osob_w_wieku_przedprodukcyjnym",
        "ludnosc_w_wieku_poprodukcyjnym_na_100_osob_w_wieku_produkcyjnym",
        "wspolczynnik_obciazenia_demograficznego_osobami_starszymi",
        "wspolczynnik_feminizacji_ogolem",
        "migracje_miedzywojewodzkie_na_pobyt_staly_wymeldowania_osob_w_wieku_20-40_lat",
        "migracje_miedzywojewodzkie_na_pobyt_staly_zameldowania_osob_w_wieku_20-40_lat",
        "zgony_wg_plci_i_grup_wieku_ogolem",
        "zgony_mezczyzni_ogolem",
        "zgony_kobiety_ogolem",
        "urodzenia_zywe_na_1000_ludnosci",
        "zgony_na_1000_ludnosci",
        "przyrost_naturalny_na_1000_ludnosci",
        "dzietnosc",
        "reprodukcja_brutto",
        "bezrobotni_zarejestrowani_pozostajacy_bez_pracy_dluzej_niz_1_rok_ogolem",
        "bezrobotni_zarejestrowani_pozostajacy_bez_pracy_dluzej_niz_1_rok_ogolem_w_%_bezrobotnych_ogolem",
        "bezrobotni_zarejestrowani_pozostajacy_bez_pracy_dluzej_niz_1_rok_mezczyzni_w_%_bezrobotnych_mezczyzn",
        "bezrobotni_zarejestrowani_pozostajacy_bez_pracy_dluzej_niz_1_rok_kobiety_w_%_bezrobotnych_kobiet"
    ],
    "values": [
        60559,
        458603,
        1645341,
        1725015,
        72305,
        72300,
        72295,
        746289,
        746290,
        746291,
        60563,
        60564,
        60562,
        634067,
        450542,
        1723482,
        1723483,
        3231,
        30149,
        30165,
        450540,
        450541,
        450551,
        59049,
        59048,
        79226,
        79228,
        453821,
        453822
    ]
}

single_frames = []
my_params = {'lang': 'pl', 'format': 'json', 'var-id': None}
for area, code in zip(data['Area'], data['Code']):
    area_frames = f"{area}".replace(" ", "_")
    locals()[area_frames] = []
    for category_code, value in zip(data2['category_code'], data2['values']):
        my_params['var-id'] = value
        response = rq.get(f'https://bdl.stat.gov.pl/api/v1/data/by-unit/{code}', params=my_params)
        output = response.json()
        df_name = f"{category_code}_{area}".replace(" ", "_")
        locals()[df_name] = pd.DataFrame(output['results'][0]['values'], columns=['year', 'val'])
        locals()[df_name] = locals()[df_name].rename(columns={"val":f"{category_code}"})
        locals()[area_frames].append(locals()[df_name])
    single_frames.append(locals()[area_frames])


In [53]:
frames = {}
for index, area in enumerate(single_frames):
    wframe = pd.DataFrame()
    for frame in area:
        if wframe.empty:
            wframe = frame
        else:
            wframe = wframe.merge(frame, on='year', how='outer')
    
    area_name = data["Area"][index]
    frames[area_name] = wframe

In [54]:
print(frames)

{'Polska':     year  ludnosc_na_1_km2  zmiana_liczby_ludnosci_na_1000_mieszkancow  \
0   1995               NaN                                         NaN   
1   1996               NaN                                         NaN   
2   1997               NaN                                         NaN   
3   1998               NaN                                         NaN   
4   1999               NaN                                         NaN   
5   2000               NaN                                         NaN   
6   2001               NaN                                         NaN   
7   2002             122.2                                         NaN   
8   2003             122.1                                        -0.7   
9   2004             122.1                                        -0.4   
10  2005             122.0                                        -0.4   
11  2006             121.9                                        -0.8   
12  2007             121.9 